In [ ]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests

In [ ]:
# Read recipe final text file, index_col=[0] is used to eliminate extra index column
df_words = pd.read_csv("recipe_final.txt",index_col=[0])

In [ ]:
# Read raw recipe file and leave only name and id column
df_raw = pd.read_csv("Resources/RAW_recipes.csv")
df_raw = df_raw[['name','id']]

In [ ]:
# Filter recipes that only have more than 300 reviews
df_words = df_words.sort_values('review', ascending = False)
df_words_filtered = df_words.loc[df_words['review'] > 300]

In [ ]:
# Merge 2 dataframes
df_words_filtered = pd.merge(df_words_filtered,df_raw, how="inner",on="id")

In [ ]:
# Leave 
scrape_final = df_words_filtered[['id','name']]
scrape_final["id"] = scrape_final["id"].apply(str)
scrape_final["recipe_url"] = ""

In [ ]:
# For each recipe id, create a recipe url to query and save in dataframe
for x in range(len(scrape_final)):
    recipe_id = scrape_final['id'][x]
    website_url = f"https://www.food.com/recipe-{recipe_id}"
    scrape_final["recipe_url"][x] = website_url
    scrape_final

In [ ]:
# url_list = [x.replace('  ', '') for x in url_list]
# url_list = [x.replace(' ', '-') for x in url_list]
# len(url_list)

In [ ]:
# Set up executable path
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=True)

In [ ]:
# Scrape the picture of the first recipe and save
json_output = []
for index,item in enumerate(scrape_final["recipe_url"]):
    try:
        print(index, item)
        time.sleep(7)
        browser.visit(item)
        time.sleep(7)
        website_html = browser.html
        website_soup = BeautifulSoup(website_html, "html.parser")
        website_results = website_soup.find("div", class_="inner-wrapper")
        print(website_results)
        picture_url = website_results.img["data-src"]
        recipe_id = scrape_final["id"][index]
        json_output.append({recipe_id: {"name": scrape_final["name"][index],"recipe_img": picture_url}})
        
    except:
        print("*** Review URL ***")
        pass

In [ ]:
json_output

In [ ]:
# Save output as json file
import json
with open('recipe_url.json', 'w') as f:
    json.dump(json_output, f, indent=4)

In [ ]:
# # Scrape the picture of the first recipe and save
# for recipe in url_list:
#     for item in range(len(scrape_final)):
#         try:
#             print(recipe)
#             time.sleep(7)
#             browser.visit(recipe)
#             time.sleep(7)
#             website_html = browser.html
#             website_soup = BeautifulSoup(website_html, "html.parser")
#             website_results = website_soup.find("div", class_="inner-wrapper")
#             print(website_results)
#             picture_url = website_results.img["data-src"]
# #             picture_list.append(picture_url)
#             json_output.append({scrape_final["id"][item]: {"name": scrape_final["name"][item]},"recipe_img": picture_url})
#         except:
#             print("*** Review URL ***")
#             print(recipe)
#             pass

In [ ]:
# recipeid, name, recipe_url, recipe image url

# json = [{recipeid: {name: brocolli,
#              image: image_url},
#  {recipeid: {name: brocolli,
#              image: image_url},]